# Customer Retention Classifier

### Customer Retention Classifier

This will be CS 570's midterm assignment. For the midterm, we will be using everything you've learned so far to build a customer "churn" model.

Imagine you work for GreenPlate, the west coast's fastest growing new subscription box service. GreenPlate's customers get a monthly box, full of healthy and sometimes delicious snacks. GreenPlate wants to know which customers are most likely to cancel their subscription service so that they can use incentives to retain those customers.

You need to use the training dataset given to create a model that predicts if the customers in the testing dataset will cancel their subscription to GreenPlate.

The dataset contains multiple features, and you can use any of those features you want. The names of those features have been hidden from you.

The dependent variable is y. If y=0 that means the customer didn't cancel. If y=1 that means the customer DID cancel. You should predict the probability of cancellation because your model will be evaluated using AUC.

You have two separate deliverables, listed below:

1. Please submit your answer via Kaggle. You'll need to be on the leaderboard to receive credit for this assignment.

2. Please submit all the code you used for cleaning, prepping, and modeling your data via blackboard (NOT github).
<br>
<br>

In [241]:
import math
import re
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_auc_score

In [242]:
pd.set_option("display.max_columns", 50)
pd.options.display.max_columns

50

In [243]:
TRAIN_DATA = pd.read_csv("../../../../data/midterm_train.csv")
TEST_DATA = pd.read_csv("../../../../data/midterm_test.csv")
X = TRAIN_DATA
y = X.pop('y')
X_combined = pd.concat([X, TEST_DATA])

In [244]:
X_combined.shape

(200000, 50)

In [245]:
X_combined.dtypes

x0     float64
x1     float64
x2     float64
x3     float64
x4     float64
x5     float64
x6     float64
x7     float64
x8     float64
x9      object
x10    float64
x11    float64
x12    float64
x13    float64
x14    float64
x15    float64
x16     object
x17    float64
x18    float64
x19     object
x20    float64
x21    float64
x22    float64
x23    float64
x24    float64
x25    float64
x26    float64
x27    float64
x28    float64
x29    float64
x30    float64
x31    float64
x32    float64
x33    float64
x34    float64
x35    float64
x36    float64
x37    float64
x38    float64
x39    float64
x40    float64
x41    float64
x42    float64
x43     object
x44     object
x45    float64
x46    float64
x47    float64
x48    float64
x49    float64
dtype: object

In [246]:
X_combined.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49
0,-5.900707,0.469389,4.733068,9.223027,3.170787,0.024669,-2.055880,-2.045466,3.004170,0.02%,-4.685655,5.895884,12.235718,-17.658996,-19.322599,1.978951,asia,0.831200,1.980735,Jun,-1.347138,0.320392,-1.177156,-2.704137,-14.103989,-5.655715,5.481989,-1.161496,-3.931582,7.643024,3.421198,3.762529,8.698367,-6.727415,-10.732802,-0.728671,0.018602,-32.680226,-0.106695,3.913680,-5.335641,-3.694592,-1.008670,wednesday,$-180.16,1.236134,4.462951,4.592803,-0.034478,3.680401
1,-3.530391,4.229818,-4.619433,2.154913,5.619903,-1.955667,7.713001,-1.279395,14.941269,0.0%,-3.103170,0.982095,3.753298,7.537105,5.942761,0.088104,asia,0.022300,-5.310811,Jun,0.898115,-1.935222,-0.398599,-0.979549,-10.746822,2.507350,1.280838,3.452034,0.624896,0.426659,2.874107,18.712973,7.312360,-15.853969,-1.455398,0.236479,-3.252878,-1.289889,14.377643,6.201034,-8.490805,6.414948,5.831287,tuesday,$432.31,-6.570015,-5.308026,-5.462467,-0.229518,-2.332947
2,1.003389,-0.012547,6.250503,3.368675,1.711321,6.192718,4.166159,-0.363055,15.175681,-0.01%,-5.424805,7.969230,-16.521783,-10.213318,9.272947,1.128275,asia,0.725844,-10.495085,July,4.546223,2.019324,4.339569,-0.205385,-15.868465,-0.501901,2.002275,0.225154,-0.882617,-14.174509,-14.707616,19.006559,2.732869,-35.200541,-21.825129,0.365506,-5.381501,16.668153,-7.932493,-0.397368,-4.172583,0.371862,-7.025987,wednesday,$-119.17,-66.851394,-11.084558,-11.407072,6.395551,3.545332
3,-16.041769,-3.243507,-0.546649,-11.594427,5.111965,0.609785,-1.540136,-1.227567,-16.460535,-0.0%,5.624369,9.138195,2.538420,14.740934,10.200809,-2.862860,asia,-0.358076,2.548242,Aug,-1.207699,-1.183008,-11.578898,-2.626346,15.628244,-16.420842,-6.891503,-4.772674,-0.172465,2.928158,-11.267913,-20.615756,5.138457,20.096343,11.691170,0.442023,-0.380077,29.446823,10.975024,1.148454,1.666316,-2.920478,-2.135088,thurday,$-1309.54,0.442939,11.808383,12.151957,-2.923916,-1.471336
4,20.498575,10.766191,1.478032,-5.723594,-0.768839,-3.482515,-11.169389,-0.361478,2.194909,-0.02%,1.331710,13.616031,2.503163,-21.545901,-31.082999,0.964522,euorpe,-0.340416,-4.412438,May,-0.504193,0.198800,-1.763387,3.767706,-13.312645,-6.421953,-3.401994,-2.892450,-5.740850,7.334497,0.836239,2.748982,7.637753,-15.819318,-7.478481,0.470682,-2.954060,-10.051513,4.656588,-5.186225,-0.757455,-1.769904,-0.532896,wednesday,$-1103.32,61.136654,48.077895,49.476758,1.043061,9.927952


### Categorical Objects:
- **x9:** string; percent change? (i.e., 0.02%, -0.01%)
- **x16:** string; region (i.e., asia, europe)
- **x19:** string; month (i.e., july, august)
- **x43:** string; day of week (i.e., wednesday, tuesday)
- **x44:** string; dollar amount (i.e., DS-180.16, DS-119.17), where DS stands in for dollar sign


### Convert Percent Change to Float

In [247]:
X_combined.x9.unique()

array(['0.02%', '0.0%', '-0.01%', '-0.0%', '-0.02%', '0.01%', '0.03%',
       '-0.03%', '-0.04%', nan, '0.05%', '0.04%', '-0.05%'], dtype=object)

In [248]:
def convertPercentChange(x):
    if isinstance(x, str): return float(re.sub('%', '', x))

In [252]:
X_combined['percentChange'] = [convertPercentChange(x) for x in X_combined.x9]

In [253]:
X_combined['percentChange'].describe()

count    199963.000000
mean          0.000005
std           0.010410
min          -0.050000
25%          -0.010000
50%           0.000000
75%           0.010000
max           0.050000
Name: percentChange, dtype: float64

### Convert Month to Float

In [262]:
X_combined.x19.unique()

array(['Jun', 'July', 'Aug', 'May', 'Mar', 'Apr', 'sept.', 'Feb', 'Oct',
       'Nov', nan, 'January', 'Dev'], dtype=object)

In [263]:
def convertMonth(x):
    if isinstance(x, str):
        if x == 'Jun': return 0
        elif x == 'July': return 1
        elif x == 'Aug': return 2
        elif x == 'May': return 3
        elif x == 'Mar': return 4
        elif x == 'Apr': return 5
        elif x == 'sept.': return 6
        elif x == 'Feb': return 7
        elif x == 'Oct.': return 8
        elif x == 'Nov': return 9
        elif x == 'January': return 10
        elif x == 'Dev': return 11

In [264]:
X_combined['month'] = [convertDay(x) for x in X_combined.x19]

In [265]:
X_combined['month'].describe()

count    200000.0
mean          5.0
std           0.0
min           5.0
25%           5.0
50%           5.0
75%           5.0
max           5.0
Name: month, dtype: float64

### Convert Day of Week to Float

In [266]:
X_combined.x43.unique()

array(['wednesday', 'tuesday', 'thurday', 'friday', 'monday', nan], dtype=object)

In [267]:
def convertDay(x):
    if isinstance(x, str):
        if x == 'monday': return 0
        elif x == 'tuesday': return 1
        elif x == 'wednesday': return 2
        elif x == 'thurday': return 3
        elif x == 'friday': return 4
        else: return 5

In [268]:
X_combined['day'] = [convertDay(x) for x in X_combined.x43]

In [269]:
X_combined['day'].describe()

count    199966.000000
mean          2.036971
std           0.620899
min           0.000000
25%           2.000000
50%           2.000000
75%           2.000000
max           4.000000
Name: day, dtype: float64

### Convert Region to Float

In [270]:
X_combined.x16.unique()

array(['asia', 'euorpe', 'america', nan], dtype=object)

In [271]:
def convertRegion(x):
    if isinstance(x, str):
        if x == 'asia': return 0
        elif x == 'europe': return 1
        elif x == 'america': return 2
        else: return 3

In [272]:
X_combined['region'] = [convertRegion(x) for x in X_combined.x16]

In [273]:
X_combined['region'].describe()

count    199953.000000
mean          0.326652
std           0.849501
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           3.000000
Name: region, dtype: float64

### Convert Dollar Amount to Float

In [274]:
X_combined.x44.unique()

array(['$-180.16', '$432.31', '$-119.17', ..., '$87.09', '$829.09',
       '$362.35'], dtype=object)

In [275]:
def convertDollarAmount(x):
    if isinstance(x, str): return float(re.sub('[$]', '', x))

In [276]:
X_combined['dollarAmt'] = [convertDollarAmount(x) for x in X_combined.x44]

In [277]:
X_combined['dollarAmt'].describe()

count    199951.000000
mean          0.027069
std        1000.004508
min       -4343.800000
25%        -672.880000
50%           1.590000
75%         672.980000
max        4929.460000
Name: dollarAmt, dtype: float64